<a href="https://colab.research.google.com/github/ice4869/titanic-analysis/blob/main/%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E5%AD%98%E6%B4%BB%E9%A0%90%E6%B8%AC%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C%EF%BC%884%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

df_train = df.copy()
#欄位刪除
df_train = df_train.drop(['Ticket' , 'PassengerId' , 'Cabin'] , axis = 1)

對Title做處理

In [3]:
df_train['Title'] = df_train['Name'].map(lambda x:x.split(',')[1].split('.')[0].strip())
df_train['Title'].value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: Title, dtype: int64

In [4]:
title_Classify = {
    'Sir':'Royalty',
    'Jonkheer':'Royalty',
    'the Countess':'Royalty',
    'Lady':'Royalty',
    'Capt':'official',
    'Col':'official',
    'Major':'official',
    'Don':'official',
    'Rev':'official',
    'Dr':'official',
    'Mlle':'Miss',
    'Miss':'Miss',
    'Mme':'Mrs',
    'Mrs':'Mrs',
    'Ms':'Mrs',
    'Mr':'Mr',
    'Master':'Master'

}
df_train['Title'] = df_train['Title'].replace(title_Classify)
df_train['Title'].value_counts()

Mr          517
Miss        184
Mrs         127
Master       40
official     19
Royalty       4
Name: Title, dtype: int64

刪除Name欄位

In [5]:
df_train = df_train.drop(['Name'] , axis = 1)

對 Missvalues 做處理

In [6]:
#Embarked眾數填補缺失
df_train['Embarked'] = df_train['Embarked'].fillna(df_train['Embarked'].mode()[0])
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,S,Mr
1,1,1,female,38.0,1,0,71.2833,C,Mrs
2,1,3,female,26.0,0,0,7.9250,S,Miss
3,1,1,female,35.0,1,0,53.1000,S,Mrs
4,0,3,male,35.0,0,0,8.0500,S,Mr
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,official
887,1,1,female,19.0,0,0,30.0000,S,Miss
888,0,3,female,NaN,1,2,23.4500,S,Miss
889,1,1,male,26.0,0,0,30.0000,C,Mr


In [7]:
#Age填補missing values by group sex、title、pclass median
age_median = df_train.pivot_table(values = 'Age' , index = ['Sex' , 'Title' , 'Pclass'] , aggfunc='median')
age_median

Age
Sex    Title    Pclass      
female Miss     1       30.0
                2       24.0
                3       18.0
       Mrs      1       40.0
                2       31.5
                3       31.0
       Royalty  1       40.5
       official 1       49.0
male   Master   1        4.0
                2        1.0
                3        4.0
       Mr       1       40.0
                2       31.0
                3       26.0
       Royalty  1       43.5
       official 1       50.0
                2       46.5

In [8]:
age_median = df_train.groupby(['Sex' , 'Title' , 'Pclass'])['Age'].transform('median')
df_train['Age'] = df_train['Age'].fillna(age_median)

In [9]:
#Sex轉數字
df_train['Sex'] = df_train['Sex'].replace({'male':0 , 'female':1})

In [10]:
#Embarked轉one-hot encode
df_train = pd.get_dummies(df_train)

In [11]:
#Pclass轉one-hot encode
Pclass_encode = pd.get_dummies(df_train['Pclass'] , prefix = 'Pclass')
df_train = df_train.drop(['Pclass'] , axis = 1)

處理Family

In [12]:
def plot_corr(dataframe , var , target , **kwargs):
  row = kwargs.get('row' , None)
  col =  kwargs.get('col' , None)
  facet = sns.FacetGrid(dataframe , hue = target , aspect=4 , row = row , col = col)
  facet.map(sns.kdeplot , var , shade = True)
  facet.set(xlim = (0 , dataframe[var].max()))
  facet.add_legend()
  plt.show()

#plot_corr(df_train , var = 'Familysize' , target='Survived')  

#處理家人
def family_handle(dataframe):
  dataframe['Familysize'] = dataframe['SibSp'] +dataframe['Parch'] + 1
  dataframe['Singe'] = dataframe['Familysize'].map(lambda x:1 if x == 1 else 0)
  dataframe['SmallFamily'] = dataframe['Familysize'].map(lambda x:1 if 2 <= x <= 4 else 0 )
  dataframe['LargeFamily'] = dataframe['Familysize'].map(lambda x:1 if 5 <= x else 0 )

  dataframe['Familysize_encode'] = dataframe['Familysize'].map(lambda x:3 if 1 <= x <= 3 else (2 if 4 <= x <= 6 else 1 ) )
  
  return dataframe
df_train = family_handle(df_train)

StandardScaler

In [13]:
#標準化年齡、價錢
from sklearn.preprocessing import StandardScaler
sz = StandardScaler()
df_train['Age'] = sz.fit_transform(df_train[['Age']])
df_train['Fare'] = sz.fit_transform(df_train[['Fare']])
df_train['Familysize_encode'] = sz.fit_transform(df_train[['Familysize_encode']])


LogisticRegression

In [14]:
df_train.drop(['SibSp' , 'Parch' , 'Singe' , 'SmallFamily' , 'LargeFamily' , 'Familysize'] , axis = 1 , inplace = True)

train_X = df_train.iloc[: , 1:]
train_y = df_train['Survived']
#迴歸 & 交叉驗證
log = LogisticRegression(random_state=0 , max_iter = 3000)
scores = cross_val_score(log , train_X , train_y.values.ravel() , cv = 5 , scoring = 'accuracy')
print(scores)
print(scores.mean())

[0.82681564 0.8258427  0.79213483 0.7752809  0.85955056]
0.8159249262444291


不同模型

In [15]:
def score(model , x , y):
  return cross_val_score(model , x , y , cv = 5 , scoring = 'accuracy').mean()

In [16]:
from sklearn.linear_model import LogisticRegression , LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [17]:
log = LogisticRegression(random_state = 0 , max_iter = 1000)
logcv = LogisticRegressionCV(random_state=0 , max_iter = 3000)
svc = SVC(random_state = 0)
decisiontree = DecisionTreeClassifier()
knn = KNeighborsClassifier(n_neighbors=3)
gaussian = GaussianNB()
rf = RandomForestClassifier(n_estimators=100)
models = [log , logcv , svc , decisiontree , knn , gaussian , rf]

In [18]:
model_names = ['Logistic Regression' , 'Logistic Regression CV' , 'SVC' , 'Decision Tree' , 'KNN' , 'Gaussian Navive' , 'Random Forest']
res = []

for model , name in zip(models , model_names):
  res.append({'Model':name , 'Average Score': score(model , train_X , train_y)})

res = pd.DataFrame(res)
res.sort_values('Average Score', ascending=False)

,Model,Average Score
2,SVC,0.820413
1,Logistic Regression CV,0.817049
0,Logistic Regression,0.815925
4,KNN,0.792367
6,Random Forest,0.786762
5,Gaussian Navive,0.780008
3,Decision Tree,0.774364


In [19]:
from sklearn.model_selection import GridSearchCV
svc = SVC()
Parameters = [{
    'random_state':[0],
    'C':[1,1.5,2,2.5,3,3.5,4,4.5,5],
    'gamma':[0.01 , 0.05 , 0.1 , 0.5 , 1],
    'kernel':['rbf']},
    {
    'C':[1,2,3,4,5,6,7,8,9,10],
    'kernel':['linear']}]

clf = GridSearchCV(svc , Parameters , cv=5 , n_jobs = 6 )
clf.fit(train_X , train_y)
svc_opt = score(clf , train_X , train_y)
print(clf.best_params_)
print(svc_opt)

{'C': 1.5, 'gamma': 0.01, 'kernel': 'rbf', 'random_state': 0}
0.8114242671520934


In [20]:
rf = RandomForestClassifier()
Parameters ={
            'max_depth' : [4, 6, 8],
            'n_estimators': [100, 50, 10],
            'max_features': ['sqrt', 'auto', 'log2'],
            'min_samples_split': [2, 3, 10],
            'min_samples_leaf': [1, 3, 10],
            'bootstrap': [True, False]}
rf = GridSearchCV(rf, Parameters,cv = 5)
rf_opt = score(rf,train_X,train_y)

串流輸出內容已截斷至最後 5000 行。
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

不同模型的結果

In [21]:
new_row = pd.DataFrame({'Model':['SVC_opt','random Forest_opt'] , 'Average Score' :[svc_opt , rf_opt]})
res = pd.concat([res , new_row] , axis = 0)
res.sort_values('Average Score' , ignore_index = True , ascending = False)

,Model,Average Score
0,SVC,0.820413
1,Logistic Regression CV,0.817049
2,Logistic Regression,0.815925
3,SVC_opt,0.811424
4,random Forest_opt,0.803584
5,KNN,0.792367
6,Random Forest,0.786762
7,Gaussian Navive,0.780008
8,Decision Tree,0.774364
